In [2]:
import pandas as pd
import nltk
import numpy as np
from nltk.stem import 	WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()
porter_stemmer  = PorterStemmer()
import preprocessor as p
#from preprocessor.api import clean, tokenize, parse,set_options
import re
from string import punctuation

In [3]:
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [4]:
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.HASHTAG)

In [5]:
df = pd.read_csv("data/Constraint_Train.csv")
val_df = pd.read_csv("data/Constraint_Val.csv")
test_df = pd.read_csv("data/Constraint_Test.csv")
test_label = pd.read_csv("data/english_test_with_labels.csv")

In [6]:
print(df['tweet'])
print(df['tweet'][6416])

0       The CDC currently reports 99031 deaths. In gen...
1       States reported 1121 deaths a small rise from ...
2       Politically Correct Woman (Almost) Uses Pandem...
3       #IndiaFightsCorona: We have 1524 #COVID testin...
4       Populous states can generate large case counts...
                              ...                        
6415    A tiger tested positive for COVID-19 please st...
6416    ???Autopsies prove that COVID-19 is??� a blood...
6417    _A post claims a COVID-19 vaccine has already ...
6418    Aamir Khan Donate 250 Cr. In PM Relief Cares Fund
6419    It has been 93 days since the last case of COV...
Name: tweet, Length: 6420, dtype: object
???Autopsies prove that COVID-19 is??� a blood clot, not pneumonia, ???and ought to be fought with antibiotics??� and the whole world has been wrong in treating the ???so-called??� pandemic.


In [8]:
def preprocess(instance):
  x = instance['tweet']
  # text = text.strip('\xa0')
  x = p.clean(x)
  tokenization = nltk.word_tokenize(x)     
  tokenization = [w for w in tokenization if not w in stop_words]
#   text = ' '.join([porter_stemmer.stem(w) for w in tokenization])
#   text = ' '.join([lemmatizer.lemmatize(w) for w in tokenization])
#   text = re.sub(r'\([0-9]+\)', '', text).strip()    
  return x

def relabel(instance):
    new_l=0
    if instance ['label']=='real':
        new_l=1
    return new_l

df.apply(lambda x: preprocess(x),1)

df['label_encoded'] = df.apply(lambda x: relabel(x), 1)
val_df['label_encoded'] = val_df.apply(lambda x: relabel(x), 1)
test_df['label_encoded'] = test_label.apply(lambda x: relabel(x), 1)

In [ ]:
print(df['tweet'])

0       The CDC currently reports 99031 deaths. In gen...
1       States reported 1121 deaths a small rise from ...
2       Politically Correct Woman (Almost) Uses Pandem...
3       #IndiaFightsCorona: We have 1524 #COVID testin...
4       Populous states can generate large case counts...
                              ...                        
6415    A tiger tested positive for COVID-19 please st...
6416    ???Autopsies prove that COVID-19 is??� a blood...
6417    _A post claims a COVID-19 vaccine has already ...
6418    Aamir Khan Donate 250 Cr. In PM Relief Cares Fund
6419    It has been 93 days since the last case of COV...
Name: tweet, Length: 6420, dtype: object


In [ ]:
def remove_punctuations(sentence):
    for p in punctuation:
        sentence = sentence.replace(p,'')
    return sentence
def to_lower_case(sentence):
    return sentence.lower()



In [19]:
real = 0
fake = 0
for i in range(6420):
    if(df['label_encoded'][i]==1):
        real = real+1
    else:
        fake = fake+1
print(real)
print(fake)
real = 0
fake = 0
for i in range(2140):
    if(val_df['label_encoded'][i]==1):
        real = real+1
    else:
        fake = fake+1
print(real)
print(fake)
real = 0
fake = 0
for i in range(2140):
    if(test_df['label_encoded'][i]==1):
        real = real+1
    else:
        fake = fake+1
print(real)
print(fake)
for i in range(6420):
    if(len(df['tweet'][i])>2000):
        print(i)
        print(df['tweet'][i])

3360
3060
1120
1020
1120
1020
270
Amoxi" Capsule made in US is the only medicine that can completely cure COVID-19.
Egyptian Health Minister arrive in Italy with secret coronavirus cure.
 Dr Charles Lieber a Harvard professor has been arrested in the USA for manufacturing the coronavirus and selling it to China. A video news report from WCVB a TV station in Boston Massachusetts is shared with the Facebook post.
Mountain Matterhorn in Switzerland was lit in three colours of Indian flag as a token of gratitude to India for sending Hydroxychloroquine.
Al-Azhar Imam said he is considering exceptions on Ramadan fasting as the research says that the mouth must be kept wet to prevent COVID-19
 Japanese Nobel Tasuku Honjo says coronavirus was created in the Wuhan laboratory
A man collapses in 5 minutes after showing COVID-19 symptoms in India.
Celebrities spreading misinformation about coronavirus and the Janata curfew in India.
 Multiple posts shared repeatedly on Facebook and Twitter since M

In [ ]:
df['tweet'] = df.apply(lambda x: preprocess(x), 1)
df['tweet'] = df['tweet'].apply(remove_punctuations)
df['tweet'] = df['tweet'].apply(to_lower_case)
val_df['tweet'] = val_df.apply(lambda x: preprocess(x), 1)
val_df['tweet'] = val_df['tweet'].apply(remove_punctuations)
val_df['tweet'] = val_df['tweet'].apply(to_lower_case)
test_df['tweet'] = test_df.apply(lambda x: preprocess(x), 1)
test_df['tweet'] = test_df['tweet'].apply(remove_punctuations)
test_df['tweet'] = test_df['tweet'].apply(to_lower_case)


In [ ]:
print(df['tweet'])
print(val_df['tweet'])
print(test_df['tweet'])

0       the cdc currently reports 99031 deaths in gene...
1       states reported 1121 deaths a small rise from ...
2       politically correct woman almost uses pandemic...
3        we have 1524 testing laboratories in india an...
4       populous states can generate large case counts...
                              ...                        
6415    a tiger tested positive for covid19 please sta...
6416    autopsies prove that covid19 is a blood clot n...
6417    a post claims a covid19 vaccine has already be...
6418     aamir khan donate 250 cr in pm relief cares fund
6419    it has been 93 days since the last case of cov...
Name: tweet, Length: 6420, dtype: object
0       chinese converting to islam after realising th...
1       11 out of 13 people from the diamond princess ...
2       covid19 is caused by a bacterium not virus and...
3       mike pence in rnc speech praises donald trumps...
4       610 skys explains the latest data and governme...
                              .

In [ ]:
all_tweets=pd.concat([df['tweet'],val_df['tweet'],test_df['tweet']])
print(all_tweets.shape)

(10700,)


In [ ]:
all_tweets[0]=all_tweets[0].astype("str")
cv=CountVectorizer(lowercase = True,ngram_range=(1,1))
cv_mixed = cv.fit_transform(all_tweets)

In [ ]:
print(all_tweets)
#print(cv_mixed)

0       the cdc currently reports 99031 deaths in gene...
1       states reported 1121 deaths a small rise from ...
2       politically correct woman almost uses pandemic...
3        we have 1524 testing laboratories in india an...
4       populous states can generate large case counts...
                              ...                        
2135     statewise details of total confirmed cases ti...
2136    tonight 12midnight onwards disaster management...
2137    296 new cases of  plateau85 enugu46 oyo31 lago...
2138    rt   onestop shop for resources has a section ...
2139    more than half of pregnant women recently admi...
Name: tweet, Length: 10700, dtype: object


In [ ]:
print(df.shape)
print(val_df.shape)
print(test_df.shape)

(6420, 4)
(2140, 4)
(2140, 2)


In [ ]:
train_sentences = cv_mixed[:6420]
val_sentences = cv_mixed[6420:(6420+2140)]
test_sentences = cv_mixed[(6420+2140):]

train_labels = df['label_encoded']
val_labels = val_df['label_encoded']
test_labels = test_df['label_encoded']

In [ ]:
def train(train_data,train_result):
    lr_model = LogisticRegression(penalty = 'l2', max_iter=200, C=1,random_state = 1)
    lr_model_train = lr_model.fit(train_data, train_result)
    lr_model_test = lr_model.predict(train_data)
    lr_model_score = accuracy_score(train_result,lr_model_test)
    return lr_model_score

def test(train_data,train_result,test_data,test_result):
    lr_model = LogisticRegression(penalty = 'l2', max_iter=200, C=1,random_state = 1)
    lr_model_train = lr_model.fit(train_data, train_result)
    lr_model_test = lr_model.predict(test_data)
    lr_model_score = accuracy_score(test_result,lr_model_test)
    return lr_model_score

In [ ]:
train_score = train(train_sentences,train_labels)
print(train_score)

0.9914330218068536


In [ ]:
val_score = test(train_sentences,train_labels,val_sentences,val_labels)
print(val_score)

0.9280373831775701


In [ ]:
test_score = test(train_sentences,train_labels,test_sentences,test_labels)
print(test_score)

0.9205607476635514
